## Solving the Diet problem in chapter 2 of the AMPL book
Minimize: <br>
3.19 XBEEF + 2.59 XCHK + 2.29 XFISH + 2.89 X HAM + <br>
1.89 X MCH + 1.99 X MTL + 1.99 XSPG + 2.49 XTUR <br>
Subject to: <br>
60 XBEEF + 8 XCHK + 8 XFISH + 40 X HAM + <br>
15 X MCH + 70 X MTL + 25 XSPG + 60 XTUR ≥ 700 <br>
20 XBEEF + 0 XCHK + 10 XFISH + 40 X HAM + <br>
35 X MCH + 30 X MTL + 50 XSPG + 20 XTUR ≥ 700 <br>
10 XBEEF + 20 XCHK + 15 XFISH + 35 X HAM + <br>
15 X MCH + 15 X MTL + 25 XSPG + 15 XTUR ≥ 700 <br>
15 XBEEF + 20 XCHK + 10 XFISH + 10 X HAM + <br>
15 X MCH + 15 X MTL + 15 XSPG + 10 XTUR ≥ 700 <br>
XBEEF ≥ 0, XCHK ≥ 0, XFISH ≥ 0, X HAM ≥ 0, <br>
X MCH ≥ 0, X MTL ≥ 0, XSPG ≥ 0, XTUR ≥ 0

In [14]:
from amplpy import AMPL
ampl = AMPL()
ampl.option['solver'] = 'cplex'

In [17]:
ampl.reset()

ampl.eval(
'''
set FOOD;
set NUTR;

param cost {FOOD} > 0;
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];
param n_min {NUTR} >= 0;
param n_max {i in NUTR} >= n_min[i];
param amt {FOOD, NUTR} >= 0;

var Buy {j in FOOD} integer >= f_min[j], <= f_max[j];

minimize Total_Cost: sum {j in FOOD} cost[j] * Buy[j];

subject to Diet {i in NUTR}:
    n_min[i] <= sum {j in FOOD} amt[j,i] * Buy[j] <= n_max[i];
'''
)

ampl.set['FOOD'] = ['BEEF', 'CHK', 'FISH', 'HAM', 'MCH', 'MTL', 'SPG', 'TUR']
ampl.set['NUTR'] = ['A', 'C', 'B1', 'B2', 'NA', 'CAL']
ampl.param['cost'] = {'BEEF': 3.19, 'CHK': 2.59, 'FISH': 2.29, 'HAM': 2.89, 'MCH': 1.89, 'MTL': 1.99, 'SPG': 1.99, 'TUR': 2.49}
ampl.param['f_min'] = {'BEEF': 2, 'CHK': 2, 'FISH': 2, 'HAM': 2, 'MCH': 2, 'MTL': 2, 'SPG': 2, 'TUR': 2}
ampl.param['f_max'] = {'BEEF': 10, 'CHK': 10, 'FISH': 10, 'HAM': 10, 'MCH': 10, 'MTL': 10, 'SPG': 10, 'TUR': 10}
ampl.param['n_min'] = {'A': 700, 'C': 700, 'B1': 700, 'B2': 700, 'NA': 0, 'CAL': 16000}
ampl.param['n_max'] = {'A': 20000, 'C': 20000, 'B1': 20000, 'B2': 20000, 'NA': 50000, 'CAL': 24000}
ampl.param['amt'] = {
    ('BEEF', 'A'): 60, ('BEEF', 'C'): 20, ('BEEF', 'B1'): 10, ('BEEF', 'B2'): 15, ('BEEF', 'NA'): 938, ('BEEF', 'CAL'): 295,
    ('CHK', 'A'): 8, ('CHK', 'C'): 0, ('CHK', 'B1'): 20, ('CHK', 'B2'): 20, ('CHK', 'NA'): 2180, ('CHK', 'CAL'): 770,
    ('FISH', 'A'): 8, ('FISH', 'C'): 10, ('FISH', 'B1'): 15, ('FISH', 'B2'): 10, ('FISH', 'NA'): 945, ('FISH', 'CAL'): 440,
    ('HAM', 'A'): 40, ('HAM', 'C'): 40, ('HAM', 'B1'): 35, ('HAM', 'B2'): 10, ('HAM', 'NA'): 278, ('HAM', 'CAL'): 430,
    ('MCH', 'A'): 15, ('MCH', 'C'): 35, ('MCH', 'B1'): 15, ('MCH', 'B2'): 15, ('MCH', 'NA'): 1182, ('MCH', 'CAL'): 315,
    ('MTL', 'A'): 70, ('MTL', 'C'): 30, ('MTL', 'B1'): 15, ('MTL', 'B2'): 15, ('MTL', 'NA'): 896, ('MTL', 'CAL'): 400,
    ('SPG', 'A'): 25, ('SPG', 'C'): 50, ('SPG', 'B1'): 25, ('SPG', 'B2'): 15, ('SPG', 'NA'): 1329, ('SPG', 'CAL'): 370,
    ('TUR', 'A'): 60, ('TUR', 'C'): 20, ('TUR', 'B1'): 15, ('TUR', 'B2'): 10, ('TUR', 'NA'): 1397, ('TUR', 'CAL'): 450
}

ampl.solve()

CPLEX 22.1.1.0: optimal integer solution; objective 119.3
9 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 1.42109e-14, relmipgap = 1.19119e-16


In [18]:
ampl.display('Buy')

ampl.display('Diet.lb', 'Diet.body', 'Diet.ub')

Buy [*] :=
BEEF   9
 CHK   2
FISH   2
 HAM   8
 MCH  10
 MTL  10
 SPG   7
 TUR   2
;

:   Diet.lb Diet.body Diet.ub    :=
A       700     2037    20000
B1      700      945    20000
B2      700      700    20000
C       700     1560    20000
CAL   16000    19155    24000
NA        0    49793    50000
;

